In [1]:
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

from config import shuffled_csv, path_exps
from NN import NN_model, Sigmoid, MSE, L2_reg
from NN.utility import batch_train, batch_out, Model_Wrapper
from LBFGS import LBFGS
from testing import multi_run

data = pd.read_csv(shuffled_csv, index_col=0).to_numpy()
data = data[:100, :]
n_samples = data.shape[0]
X_data = data[:, :10]
Y_data = data[:, 10:]

Y_scaler = StandardScaler()
Y_scaled = Y_scaler.fit_transform(Y_data)

# np.random.seed(11)
model = NN_model([10, 20, 20, 2], Sigmoid, MSE)
model.init_weights()
reg_loss = L2_reg(1e-4)

# logging.basicConfig(level="INFO")
f = Model_Wrapper(model, X_data, Y_scaled, reg_loss)
x = model.Weights

In [2]:
import seaborn as sns
from adam import adam_SGD

def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod() ** (1.0 / len(a))

def pair_vector(vector_list):
    maxlen = len(max(vector_list, key=len))
    for i in range(len(vector_list)):
        lastEl = len(vector_list[i]) - 1
        tmpArray = np.full((maxlen - lastEl - 1, 1), vector_list[i][lastEl])
        vector_list[i] = np.array(np.concatenate((vector_list[i], tmpArray), axis=None))
    return vector_list

runs = 5
print('Testing ADAM')


adam_f_values = []
lbfgs_f_values = []
for i in range(runs):
    solver = adam_SGD(max_feval = 23)
    model.init_weights()
    x = model.Weights
    status = solver.solve(f, x)
    f_values = solver.f_values
    adam_f_values.append(f_values)

for i in range(runs):
    solver = LBFGS(eps=1e-20,max_feval=5,M=200000)
    model.init_weights()
    x = model.Weights
    status = solver.solve(f, x)
    f_values = solver.f_values
    lbfgs_f_values.append(f_values)

print(adam_f_values)
print(lbfgs_f_values)

if lbfgs_f_values is not None and adam_f_values is not None: 
    run_1 = pair_vector(adam_f_values)
    run_2 = pair_vector(lbfgs_f_values)
    df_1 = pd.DataFrame(run_1).melt()
    df_1['function'] = ["adam"] * len(df_1.index)
    df_2 = pd.DataFrame(run_2).melt()
    df_2['function'] = ["lbfgs"] * len(df_2.index)
    result = pd.concat([df_1, df_2], axis=0)
    
    # Plot fevals
    fig = sns.lineplot(x="variable", y="value", hue="function", data=result, estimator=geo_mean)
    fig.set_yscale("log")
    fig.set(xlabel='f_evaluations', ylabel='Model Loss')

Testing ADAM
[[187.79879213172086, 178.89749766689692, 168.5810851594451, 157.45556101884728, 145.6966137623854, 133.46726097757164, 120.96917640937808, 108.47431302713963, 96.34894707332987, 85.05621304926507, 75.13143300118382, 67.12084023142343, 61.463421689703644, 58.310860981808666, 57.324271782459405, 57.60229596885128, 57.940920040721345, 57.327853819967935, 55.2814637468316, 51.85715645240378, 47.46333345173251, 42.65536460800763, 37.979215505637754, 33.873969696329425], [235.7397622678575, 222.62038855042434, 208.13037178402737, 193.4068142994962, 178.75816589712392, 164.2846186979425, 150.07410664460016, 136.24600406799985, 122.95090502453657, 110.36996838796031, 98.71460488658221, 88.2175904721937, 79.10868224773657, 71.57430448198559, 65.70944156224677, 61.47171119683368, 58.64833908444048, 56.85582209197456, 55.599496545133135, 54.391766230820984, 52.869132782425275, 50.846205087630224, 48.30227163256272, 45.335571719815704], [204.58606832211743, 195.58933490212294, 185.52

IndexError: list index out of range